In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import tensorflow as tf

In [2]:
con = sqlite3.connect("database.sqlite")

In [92]:
df = pd.read_sql_query("SELECT * FROM Reviews", con)
df.index = df["Id"]
df["Text"][2]

'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".'

In [94]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
Id,,,,,,,,,,
1,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
5,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568450,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568451,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568452,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."


In [95]:
df["Text"][3]

'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.'

In [4]:
df.pop("Id")

Id
1              1
2              2
3              3
4              4
5              5
           ...  
568450    568450
568451    568451
568452    568452
568453    568453
568454    568454
Name: Id, Length: 568454, dtype: int64

In [5]:
df.columns

Index(['ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
dftrain, dftest = train_test_split(df, test_size = 0.2)

In [7]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
    global word_encoding

    words = text.lower().split(" ") 
    encoding = []  

    for word in words:
        if word in vocab:
            code = vocab[word]  
            encoding.append(code) 
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            word_encoding += 1
    return encoding

In [8]:
dftrain["encoding"] = [one_hot_encoding(dftrain.iloc[i,8]) for i in range(454763)]

<ipython-input-8-ae9b623f06b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftrain["encoding"] = [one_hot_encoding(dftrain.iloc[i,8]) for i in range(454763)]


In [9]:
dftest["encoding"] = [one_hot_encoding(dftrain.iloc[i,8]) for i in range(113691)]

<ipython-input-9-d17ab49e3f69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftest["encoding"] = [one_hot_encoding(dftrain.iloc[i,8]) for i in range(113691)]


In [10]:
dftrainnew = dftrain["encoding"]
dftestnew = dftest["encoding"]

In [51]:
dftrain_labels = dftrain["Score"]
dftest_labels = dftest["Score"]
len(dftest_labels)

113691

In [52]:
ar = []
for i in dftrain_labels.values:
    ar.append(i)
dftrain_labels_encoding = np.array(ar)

ar1 = []
for i in dftest_labels.values:
    ar1.append(i)
dftest_labels_encoding = np.array(ar1)

In [53]:
dftrainnew_encoding = np.asarray(dftrainnew.values.tolist())
dftestnew_encoding = np.asarray(dftestnew.values.tolist())

<ipython-input-53-57c48caa876f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dftrainnew_encoding = np.asarray(dftrainnew.values.tolist())
<ipython-input-53-57c48caa876f>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dftestnew_encoding = np.asarray(dftestnew.values.tolist())


In [54]:
dftrainnew_encoding

array([list([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 16, 3, 10, 22, 23, 24, 25, 26, 27, 2, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 25, 10, 45, 36, 46, 47, 48, 49, 50, 51, 52, 53]),
       list([54, 4, 55, 25, 56, 57, 58, 59, 60, 61, 62, 63, 64, 63, 65, 66, 67, 7, 10, 68, 69, 17, 70, 63, 65, 71, 46, 72, 61, 73, 74, 75, 46, 10, 76, 77, 78, 70, 79, 80, 81, 82, 83, 10, 84, 52, 85, 71, 46, 72, 61, 13, 86, 38, 87, 70, 1, 88, 89, 17, 44, 25, 10, 90, 70, 91, 92, 93, 94, 95, 96, 7, 97, 98, 99, 37, 100, 101, 52, 102, 103, 104, 105, 106, 95, 107, 108, 57, 100, 109, 110, 111, 33, 112, 29, 113, 114, 23, 113, 115, 114, 116, 117]),
       list([47, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 38, 129, 130, 131, 20, 132, 133, 134, 135, 136, 127, 137, 4, 138, 25, 123, 124, 139, 16, 140, 141, 127, 142, 143, 144, 134, 46, 145, 146, 127, 17, 147, 148, 149, 134, 150, 25, 145, 151, 13, 86, 148, 152, 95, 153, 41, 10, 154, 7, 101, 155, 1

In [55]:
len(dftestnew_encoding)

113691

In [56]:
dftrain_labels_encoding

array([4, 1, 4, ..., 5, 5, 3], dtype=int64)

In [57]:
len(dftest_labels_encoding)

113691

In [69]:
dftrainnew_encoding = tf.keras.preprocessing.sequence.pad_sequences(dftrainnew_encoding, 250)
dftestnew_encoding = tf.keras.preprocessing.sequence.pad_sequences(dftestnew_encoding, 250)
dftestnew_encoding 

array([[    0,     0,     0, ...,    51,    52,    53],
       [    0,     0,     0, ...,   114,   116,   117],
       [  134,   150,    25, ...,   266,    41,   267],
       ...,
       [    0,     0,     0, ...,   113, 31431, 21413],
       [    0,     0,     0, ...,  2737,    65,  2255],
       [    0,     0,     0, ...,  1563,   104,  5775]])

In [65]:
dftrainlabels = tf.one_hot(dftrain_labels_encoding, 6)
dftestlabels = tf.one_hot(dftest_labels_encoding, 6)
dftestlabels

<tf.Tensor: shape=(113691, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [37]:
VOCAB_SIZE = 500000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 100),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(6, activation="sigmoid")
])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         50000000  
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
Total params: 50,118,022
Trainable params: 50,118,022
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])

In [41]:
history = model.fit( tf.convert_to_tensor(dftrainnew_encoding),  tf.convert_to_tensor(dftrainlabels), batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
2843/2843 [==============================] - 3671s 1s/step - loss: 0.7379 - acc: 0.7301 - val_loss: 0.6275 - val_acc: 0.7700
Epoch 2/5
2843/2843 [==============================] - 33449s 12s/step - loss: 0.4800 - acc: 0.8250 - val_loss: 0.5895 - val_acc: 0.7969
Epoch 3/5
2843/2843 [==============================] - 3981s 1s/step - loss: 0.3070 - acc: 0.8903 - val_loss: 0.6304 - val_acc: 0.7997
Epoch 4/5
2843/2843 [==============================] - 8470s 3s/step - loss: 0.1866 - acc: 0.9346 - val_loss: 0.7550 - val_acc: 0.7929
Epoch 5/5
2843/2843 [==============================] - 4466s 2s/step - loss: 0.1164 - acc: 0.9597 - val_loss: 0.8917 - val_acc: 0.8005


In [70]:
results = model.evaluate(tf.convert_to_tensor(dftestnew_encoding), dftestlabels)
print(results)

3553/3553 [==============================] - 273s 77ms/step - loss: 5.1597 - acc: 0.4490
[5.159664154052734, 0.4489537477493286]


In [72]:
def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [vocab[word] if word in vocab else 0 for word in tokens]
    return tf.keras.preprocessing.sequence.pad_sequences([tokens], 250)[0]
encode_text("Awesome service")

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [93]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred) 
    return max(result[0]), list(result[0]).index(max(result[0]))
predict("Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as Jumbo.")

1/1 [==============================] - 0s 33ms/step


(0.86436886, 1)

In [96]:
predict("This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' The Lion, The Witch, and The Wardrobe - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.")

1/1 [==============================] - 0s 34ms/step


(0.91233927, 4)

In [97]:
predict("Hell")

1/1 [==============================] - 0s 36ms/step


(0.99311304, 0)

In [102]:
predict("Stale")

1/1 [==============================] - 0s 36ms/step


(0.9986723, 0)

In [104]:
predict("Good")

1/1 [==============================] - 0s 38ms/step


(0.85214335, 4)